In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import torch
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using{device}')



/kaggle/input/open-problems-single-cell-perturbations/multiome_train.parquet
/kaggle/input/open-problems-single-cell-perturbations/multiome_obs_meta.csv
/kaggle/input/open-problems-single-cell-perturbations/sample_submission.csv
/kaggle/input/open-problems-single-cell-perturbations/adata_train.parquet
/kaggle/input/open-problems-single-cell-perturbations/multiome_var_meta.csv
/kaggle/input/open-problems-single-cell-perturbations/adata_obs_meta.csv
/kaggle/input/open-problems-single-cell-perturbations/id_map.csv
/kaggle/input/open-problems-single-cell-perturbations/de_train.parquet
/kaggle/input/open-problems-single-cell-perturbations/adata_excluded_ids.csv
Usingcuda


# Data Processing

In [4]:
df = pd.read_parquet('../input/open-problems-single-cell-perturbations/de_train.parquet')
df.tail()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
609,T regulatory cells,Atorvastatin,LSM-5771,CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...,False,-0.014372,-0.122464,-0.456366,-0.147894,-0.545382,...,-0.549987,-2.200925,0.359806,1.073983,0.356939,-0.029603,-0.528817,0.105138,0.491015,-0.979951
610,NK cells,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,-0.455549,0.188181,0.595734,-0.100299,0.786192,...,-1.236905,0.003854,-0.197569,-0.175307,0.101391,1.028394,0.034144,-0.231642,1.023994,-0.064760
611,T cells CD4+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.338168,-0.109079,0.270182,-0.436586,-0.069476,...,0.077579,-1.101637,0.457201,0.535184,-0.198404,-0.005004,0.552810,-0.209077,0.389751,-0.337082
612,T cells CD8+,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,0.101138,-0.409724,-0.606292,-0.071300,-0.001789,...,0.005951,-0.893093,-1.003029,-0.080367,-0.076604,0.024849,0.012862,-0.029684,0.005506,-1.733112
613,T regulatory cells,Riociguat,LSM-45758,COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...,False,-0.757116,0.085910,-0.730025,-1.367801,-0.695944,...,0.232343,-2.247816,-0.346036,-0.919567,-1.131372,-0.120252,-0.064537,-0.603280,-0.098041,-0.750681


In [5]:
column_array = df['SMILES'].values

In [6]:
##column_array

# Data Loader

# Building Experts

# final trial

In [6]:
feature_cols =['cell_type','sm_name']
target_cols = ['cell_type','sm_name','sm_lincs_id','SMILES','control']
targets = df.drop(columns=target_cols)
features = pd.DataFrame(df,columns=feature_cols)
features

,cell_type,sm_name
0,NK cells,Clotrimazole
1,T cells CD4+,Clotrimazole
2,T cells CD8+,Clotrimazole
3,T regulatory cells,Clotrimazole
4,NK cells,Mometasone Furoate
...,...,...
609,T regulatory cells,Atorvastatin
610,NK cells,Riociguat
611,T cells CD4+,Riociguat
612,T cells CD8+,Riociguat


In [7]:
from sklearn.preprocessing import OneHotEncoder
one_hot = OneHotEncoder()
features_array = one_hot.fit_transform(features)

In [8]:
features_array.shape

(614, 152)

In [9]:
features_one_hot=features_array
print(features_one_hot.toarray().shape)


(614, 152)


In [10]:
from sklearn.model_selection import train_test_split

# Split the data into 70% training, 15% validation, and 15% testing
X_train, X_temp, y_train, y_temp = train_test_split(features_one_hot, targets.values, test_size=0.1, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

In [11]:
featurespace = features_one_hot.toarray()
targetsspace = targets.values
targetsspace.shape

(614, 18211)

In [12]:
from torch.utils.data import DataLoader,Dataset
class dataset(Dataset):
    def __init__(self,X_train,y_train):
        self.X_train=X_train
        self.y_train=y_train
    def __len__(self):
        return len(self.X_train)
    def __getitem__(self,idx):
        return self.X_train[idx], self.y_train[idx]

In [13]:
import torch

features_t= torch.tensor(featurespace).to(device)
target_t=torch.tensor(targetsspace).to(device)
data=dataset(features_t,target_t)
target_t.shape

torch.Size([614, 18211])

In [14]:
batchsize=64
dataloader = torch.utils.data.DataLoader(data,batch_size=batchsize,shuffle=True)
for batch,(x,y) in enumerate(dataloader):
    print('batch',batch,x.shape,y.shape)

batch 0 torch.Size([64, 152]) torch.Size([64, 18211])
batch 1 torch.Size([64, 152]) torch.Size([64, 18211])
batch 2 torch.Size([64, 152]) torch.Size([64, 18211])
batch 3 torch.Size([64, 152]) torch.Size([64, 18211])
batch 4 torch.Size([64, 152]) torch.Size([64, 18211])
batch 5 torch.Size([64, 152]) torch.Size([64, 18211])
batch 6 torch.Size([64, 152]) torch.Size([64, 18211])
batch 7 torch.Size([64, 152]) torch.Size([64, 18211])
batch 8 torch.Size([64, 152]) torch.Size([64, 18211])
batch 9 torch.Size([38, 152]) torch.Size([38, 18211])


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np
from scipy.sparse import issparse

# Define custom loss function
class LogCoshLoss(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, y_prime_t, y_t):
        ey_t = (y_t - y_prime_t) / 3  # divide by 3 to avoid numerical overflow in cosh
        return torch.mean(torch.log(torch.cosh(ey_t + 1e-12)))

# Define expert models
class Conv(nn.Module):
    def __init__(self, scheme, input_channels=1):
        super(Conv, self).__init__()
        self.name = 'Conv'
        self.conv_block = nn.Sequential(
            nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Conv1d(32, 64, kernel_size=5, stride=2, padding=2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Conv1d(64, 128, kernel_size=5, stride=2, padding=2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten())
        self.linear = nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.head1 = nn.Linear(512, output_size)
        
    def forward(self, x):
        # Ensure input is 3D: (batch_size, channels, sequence_length)
        if x.dim() == 2:
            x = x.unsqueeze(1)
        out = self.conv_block(x)
        out = self.linear(out)
        out = self.head1(out)
        return out

class LSTM(nn.Module):
    def __init__(self, scheme, input_size, output_size):
        super(LSTM, self).__init__()
        self.name = 'LSTM'
        self.lstm = nn.LSTM(input_size, 128, num_layers=2, batch_first=True)
        self.linear = nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.head1 = nn.Linear(512, output_size)
        
    def forward(self, x):
        # Ensure input is 3D: (batch_size, sequence_length, input_size)
        if x.dim() == 2:
            x = x.unsqueeze(1)  # Add sequence dimension
        
        batch_size, seq_len, _ = x.size()
        
        out, (hn, cn) = self.lstm(x)
        
        if seq_len == 1:
            out = out.squeeze(1)  # Remove sequence dimension if it's 1
        else:
            out = out[:, -1, :]  # Take the last time step
        
        out = self.linear(out)
        out = self.head1(out)
        return out

class GRU(nn.Module):
    def __init__(self, scheme, input_size, output_size):
        super(GRU, self).__init__()
        self.name = 'GRU'
        self.gru = nn.GRU(input_size, 128, num_layers=2, batch_first=True)
        self.linear = nn.Sequential(
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.head1 = nn.Linear(512, output_size)
        
    def forward(self, x):
        # Ensure input is 3D: (batch_size, sequence_length, input_size)
        if x.dim() == 2:
            x = x.unsqueeze(1)  # Add sequence dimension
        
        batch_size, seq_len, _ = x.size()
        
        out, hn = self.gru(x)
        
        if seq_len == 1:
            out = out.squeeze(1)  # Remove sequence dimension if it's 1
        else:
            out = out[:, -1, :]  # Take the last time step
        
        out = self.linear(out)
        out = self.head1(out)
        return out

# Define Gating Network
class GatingNetwork(nn.Module):
    def __init__(self, input_size, num_experts):
        super(GatingNetwork, self).__init__()
        self.linear1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(64, num_experts)
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self, data):
        x = self.linear1(data)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

# Define Mixture of Experts model
class MixtureOfExperts(nn.Module):
    def __init__(self, input_size, output_size, scheme):
        super(MixtureOfExperts, self).__init__()
        self.experts = nn.ModuleList([
            Conv(scheme, input_channels=1),
            LSTM(scheme, input_size, output_size),
            GRU(scheme, input_size, output_size)
        ])
        self.gating_network = GatingNetwork(input_size, len(self.experts))
        self.loss1 = nn.MSELoss()
        self.loss2 = LogCoshLoss()
        self.loss3 = nn.L1Loss()
        self.loss4 = nn.BCELoss()
    
    def forward(self, data, y=None):
        data = data.float()
        
        gating_weights = self.gating_network(data)
        
        expert_outputs = []
        for i, expert in enumerate(self.experts):
            if i == 0:  # Conv
                expert_input = data.unsqueeze(1) if data.dim() == 2 else data
            else:  # LSTM and GRU
                expert_input = data if data.dim() == 3 else data.unsqueeze(1)
            
            output = expert(expert_input)
            expert_outputs.append(output)
        
        expert_outputs = torch.stack(expert_outputs, dim=1)
        combined_output = torch.sum(expert_outputs * gating_weights.unsqueeze(-1), dim=1)
        
        if y is None:
            return combined_output
        else:
            y = y.float()
            loss1 = 0.4 * self.loss1(combined_output, y) + 0.3 * self.loss2(combined_output, y) + 0.3 * self.loss3(combined_output, y)
            yhat = torch.sigmoid(combined_output)
            yy = torch.sigmoid(y)
            loss2 = self.loss4(yhat, yy)
            return 0.8 * loss1 + 0.2 * loss2
# Define RMSE rowwise loss
def RMSE_rowwise_loss(model, data, y_true):
    model.eval()
    with torch.no_grad():
        y_pred_original = model(data.float())
    mae = mean_absolute_error(y_true.numpy(), y_pred_original.numpy())
    rowwise_rmse = np.sqrt(np.mean(np.square(y_true.numpy() - y_pred_original.numpy()), axis=1))
    mrrmse_score = np.mean(rowwise_rmse)
    return mrrmse_score

# Usage
input_size = 152
output_size = 18211
scheme = 'initial'
epochs = 2000

model = MixtureOfExperts(input_size, output_size, scheme)
model = model.float()
model = model.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [18]:
print(model)

MixtureOfExperts(
  (experts): ModuleList(
    (0): Conv(
      (conv_block): Sequential(
        (0): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=(2,))
        (1): ReLU()
        (2): Dropout(p=0.3, inplace=False)
        (3): Conv1d(32, 64, kernel_size=(5,), stride=(2,), padding=(2,))
        (4): ReLU()
        (5): Dropout(p=0.3, inplace=False)
        (6): Conv1d(64, 128, kernel_size=(5,), stride=(2,), padding=(2,))
        (7): ReLU()
        (8): Dropout(p=0.3, inplace=False)
        (9): AdaptiveAvgPool1d(output_size=1)
        (10): Flatten(start_dim=1, end_dim=-1)
      )
      (linear): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.3, inplace=False)
        (3): Linear(in_features=512, out_features=512, bias=True)
        (4): ReLU()
        (5): Dropout(p=0.3, inplace=False)
      )
      (head1): Linear(in_features=512, out_features=18211, bias=True)
    )
    (1): LSTM(
      (lstm): LS

In [16]:
import logging
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def train_model(model, data, optimizer, epochs, batch_size, device):
    dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        total_batches = 0
        
        for batch, (x, y) in enumerate(dataloader):
            x = x.float().to(device)
            y = y.float().to(device)
            
            optimizer.zero_grad()
            try:
                loss = model(x, y)
                loss.backward()
                optimizer.step()
                
                epoch_loss += loss.item()
                total_batches += 1
            except RuntimeError as e:
                print(f"Error in epoch {epoch+1}, batch {batch+1}: {str(e)}")
                print(f"Input shapes: x: {x.shape}, y: {y.shape}")
                continue
        
        avg_loss = epoch_loss / total_batches
        print(f'Epoch {epoch+1}/{epochs} completed. Average loss: {avg_loss:.4f}')

# Usage
epochs = 2000
batch_size = 64
train_model(model, data, optimizer, epochs, batch_size, device)

Epoch 1/2000 completed. Average loss: 2.1198
Epoch 2/2000 completed. Average loss: 2.2367
Epoch 3/2000 completed. Average loss: 2.1340
Epoch 4/2000 completed. Average loss: 2.0878
Epoch 5/2000 completed. Average loss: 1.9561
Epoch 6/2000 completed. Average loss: 1.8084
Epoch 7/2000 completed. Average loss: 1.8088
Epoch 8/2000 completed. Average loss: 1.5400
Epoch 9/2000 completed. Average loss: 1.4963
Epoch 10/2000 completed. Average loss: 1.5017
Epoch 11/2000 completed. Average loss: 1.4245
Epoch 12/2000 completed. Average loss: 1.4185
Epoch 13/2000 completed. Average loss: 1.4239
Epoch 14/2000 completed. Average loss: 1.4101
Epoch 15/2000 completed. Average loss: 1.3819
Epoch 16/2000 completed. Average loss: 1.3663
Epoch 17/2000 completed. Average loss: 1.3464
Epoch 18/2000 completed. Average loss: 1.3661
Epoch 19/2000 completed. Average loss: 1.2626
Epoch 20/2000 completed. Average loss: 1.2809
Epoch 21/2000 completed. Average loss: 1.1980
Epoch 22/2000 completed. Average loss: 1.17

KeyboardInterrupt: 

In [84]:
# import torch.optim as optim
# from torch.autograd import Variable

# input_size = 152
# output_size = 18211
# num_experts = 5
# epochs = 200

# # Create the model
# model = MixtureOfExperts(input_size, output_size, num_experts)

# # Define the optimizer
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# for epoch in range(0, epochs):
#     for batch, (feature_train, target_train) in enumerate(dataloader):
#         optimizer.zero_grad()
#         loss = RMSE_rowwise_loss(model, feature_train.float(), target_train.float())
#         loss = Variable(torch.tensor(loss), requires_grad=True)
#         loss.backward()
#         optimizer.step()
#     if epoch % 5 == 0:
#         print(f"epoch: {epoch + 1} loss: {loss}")

In [19]:
id_map = pd.read_csv("/kaggle/input/open-problems-single-cell-perturbations/id_map.csv")

In [20]:
testdata = pd.DataFrame(id_map,columns=feature_cols) 
#testdata.head()

In [21]:
one_hot_test = one_hot.transform(testdata)
print(features_one_hot.toarray().shape,one_hot_test.toarray().shape)

(614, 152) (255, 152)


In [22]:
model.eval()
target_pred = model(torch.Tensor(one_hot_test.toarray()).to(device))

In [23]:
res = target_pred.cpu()

In [24]:
model.eval()

MixtureOfExperts(
  (experts): ModuleList(
    (0): Conv(
      (conv_block): Sequential(
        (0): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=(2,))
        (1): ReLU()
        (2): Dropout(p=0.3, inplace=False)
        (3): Conv1d(32, 64, kernel_size=(5,), stride=(2,), padding=(2,))
        (4): ReLU()
        (5): Dropout(p=0.3, inplace=False)
        (6): Conv1d(64, 128, kernel_size=(5,), stride=(2,), padding=(2,))
        (7): ReLU()
        (8): Dropout(p=0.3, inplace=False)
        (9): AdaptiveAvgPool1d(output_size=1)
        (10): Flatten(start_dim=1, end_dim=-1)
      )
      (linear): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.3, inplace=False)
        (3): Linear(in_features=512, out_features=512, bias=True)
        (4): ReLU()
        (5): Dropout(p=0.3, inplace=False)
      )
      (head1): Linear(in_features=512, out_features=18211, bias=True)
    )
    (1): LSTM(
      (lstm): LS

In [25]:
sample_submission = pd.read_csv("/kaggle/input/open-problems-single-cell-perturbations/sample_submission.csv")

In [26]:
sample_columns = sample_submission.columns
sample_columns= sample_columns[1:]
submission_df = pd.DataFrame(res.detach().numpy(), columns=sample_columns)

In [27]:
submission_df.insert(0, 'id', range(255))

In [28]:
sample_submission

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251,251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
252,252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253,253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
submission_df

,id,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,A4GALT,AAAS,AACS,AAGAB,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,0,0.325826,-0.007866,0.370508,0.661771,1.344700,0.952999,-0.112421,0.013561,-0.186119,...,-0.712701,-0.251880,-0.117705,0.244854,0.316382,0.361226,0.317160,0.428898,-0.035407,0.042672
1,1,0.146974,0.055501,0.330907,0.320045,0.343979,0.171514,-0.107187,0.163914,0.414388,...,0.436475,0.073368,-0.217088,0.200419,0.118749,0.008344,0.174967,0.080371,-0.462454,-0.144866
2,2,0.429456,0.060914,0.666287,1.160771,3.196780,2.379148,0.338011,1.073238,0.860730,...,-0.026625,0.435995,1.161289,1.409133,0.578077,0.798683,0.937108,0.019590,-0.292967,-0.044822
3,3,-0.044657,-0.017270,0.053793,0.155811,0.216723,0.003991,-0.174959,-0.062513,-0.023720,...,-0.192043,-0.026088,-0.167034,-0.116363,0.015246,-0.020211,0.122894,0.086638,0.061081,-0.124278
4,4,0.428918,0.016291,0.606734,0.789091,2.402305,1.698626,-0.052214,-0.135977,0.161543,...,-0.600954,0.254689,0.126396,0.555928,0.636604,0.189434,0.461295,0.149743,0.065418,0.053007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,250,-0.182730,-0.186048,-0.122282,-0.236606,1.192663,0.322482,0.159350,0.261420,0.040568,...,-0.078092,-0.921257,-1.594355,-0.888925,-0.196915,-0.390939,-0.629882,-0.303671,-1.398325,-0.638539
251,251,0.166292,0.172093,-0.264903,0.312848,2.244164,0.697194,0.063729,-0.061041,-0.336808,...,-0.447420,-0.489593,-0.637586,-0.570963,0.421161,-0.205135,-0.238691,0.157226,-0.744223,-0.297976
252,252,0.179165,0.050252,-1.559113,-0.208699,2.996405,1.089666,0.169657,-0.199713,0.011201,...,-0.272914,-0.772601,-1.286773,-0.605035,0.342517,0.162946,-0.581229,-0.558160,-0.167480,-0.208152
253,253,0.719092,1.957038,-9.073939,-1.067257,9.791842,9.566616,1.057602,-0.367063,0.284872,...,-1.418733,-1.232353,-4.137112,0.878891,2.284428,0.125582,-2.409629,-3.088445,-0.943358,-0.822677


In [30]:
submission_df.to_csv("submission.csv", index=False)